In [75]:
!uv pip show policyengine-uk

Using Python 3.11.11 environment at: /Users/nikhilwoodruff/policyengine/policyengine-app/.venv
Name: policyengine-uk
Version: 2.28.1
Location: /Users/nikhilwoodruff/policyengine/policyengine-app/.venv/lib/python3.11/site-packages
Requires: microdf-python, policyengine-core
Required-by: policyengine


Reform UK leader Nigel Farage today confirmed four tax-benefit policy reforms would be included in the party's manifesto for the next general election in a [speech](https://www.bbc.co.uk/news/articles/c5yx062pvlvo) in central London. The party had previously included three of these reforms in its 2024 general election manifesto, committing to:

* Increase the Personal Allowance from £12,570 to £20,000
* Increase the higher rate threshold from £50,270 to £70,000
* Raise the Marriage Allowance maximum rate from 10% to 25%

Today Farage confirmed that the party would include these reforms in the next general election manifesto, alongside a third commitment to repeal the two-child benefit cap on Universal Credit and Child Tax Credit payments.

## Currrent policy

### Personal Allowance

The government exempts the first £12,570 of an individual's income from income tax, known as the **Personal Allowance**. The personal allowance is gradually reduced for individuals earning over £100,000, with a complete withdrawal at £125,140. The government has frozen this threshold since 2021, and is scheduled to remain at this level until 2028.

### Marriage Allowance

The Marriage Allowance permits couples to transfer 10% of any unused personal allowance between spouses when one earns below the Personal Allowance threshold and the other pays the basic rate.

### Two-child limit

The two-child limit removes the entitlement of children born after April 2017 to Universal Credit and Child Tax Credit child element payments, if they are the third or subsequent child in a family, unless [exceptions](https://www.gov.uk/guidance/child-tax-credit-exceptions-to-the-2-child-limit#overview) apply.

### Economic impacts

PolicyEngine's model [estimates](https://policyengine.org/uk/policy?reform=85289&focus=policyOutput.budgetaryImpact&region=uk&timePeriod=2025&baseline=1&uk_local_areas_beta=false&simYears=5) that these reforms would raise the deficit by £77 billion in 2025-26, rising to £Y billion in 2029-30.

*Table 1: Estimated revenue impacts of Reform UK's reform proposals (£ billion)*
| Policy                                   |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-29 |
|:-----------------------------------------|-------:|-------:|-------:|-------:|-------:|----------:|
| Repeal the two-child limit               |   -1.7 |   -1.8 |   -1.9 |   -2.2 |   -2.3 |      -9.9 |
| Raise the Personal Allowance to £20,000  |  -50.7 |  -51.5 |  -52.4 |  -49.1 |  -45.5 |    -249.3 |
| Set the higher rate threshold to £70,000 |  -22.2 |  -23.8 |  -25.5 |  -27.2 |  -29.3 |    -128   |
| Raise the Marriage Allowance rate to 25% |   -0.7 |   -0.7 |   -0.8 |   -0.8 |   -0.9 |      -3.9 |
| Total                                    |  -75.3 |  -77.9 |  -80.5 |  -79.3 |  -78.1 |    -391.1 |


In [50]:
from policyengine import Simulation

raise_pa = {
    "gov.hmrc.income_tax.allowances.personal_allowance.amount": 20_000,
    "gov.hmrc.income_tax.rates.uk[1].threshold": 37_700 - (20_000 - 12_570),
}
higher_rate_threshold = {
    "gov.hmrc.income_tax.rates.uk[1].threshold": 50_000,
}
raise_ma_rate = {
    "gov.hmrc.income_tax.allowances.marriage_allowance.max": 0.25,
}
repeal_two_child_limit = {
    "gov.dwp.universal_credit.elements.child.limit.child_count": 90,
}

policy_reforms = [
    repeal_two_child_limit,
    raise_pa,
    higher_rate_threshold,
    raise_ma_rate,
]
policy_reform_names = [
    "Repeal the two-child limit",
    "Raise the Personal Allowance to £20,000",
    "Set the higher rate threshold to £70,000",
    "Raise the Marriage Allowance rate to 25%",
]

years = []
debt_impacts = []
policies = []

from tqdm import tqdm

for i in tqdm(range(len(policy_reforms))):
    reform = {}
    for j in range(i + 1):
        reform.update(policy_reforms[j])

    sim = Simulation(
        country="uk",
        scope="macro",
        reform=reform,
    )
    print(reform, policy_reform_names[i])

    for year in tqdm(range(2025, 2030)):
        debt_impact = sim.reform_simulation.calculate("gov_balance", year).sum()/1e9 - sim.baseline_simulation.calculate("gov_balance", year).sum()/1e9
        years.append(year)
        debt_impacts.append(debt_impact)
        policies.append(policy_reform_names[i])



  0%|          | 0/4 [00:00<?, ?it/s]INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.6 to cache
INFO:policyengine.utils.data.caching_google_storage_client:Cache exists and crc is unchanged for policyengine-uk-data-private, enhanced_frs_2022_23.h5.
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5


{'gov.dwp.universal_credit.elements.child.limit.child_count': 90} Repeal the two-child limit


 25%|██▌       | 1/4 [00:54<02:42, 54.10s/it]INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.6 to cache
INFO:policyengine.utils.data.caching_google_storage_client:Cache exists and crc is unchanged for policyengine-uk-data-private, enhanced_frs_2022_23.h5.
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5


{'gov.dwp.universal_credit.elements.child.limit.child_count': 90, 'gov.hmrc.income_tax.allowances.personal_allowance.amount': 20000, 'gov.hmrc.income_tax.rates.uk[1].threshold': 30270} Raise the Personal Allowance to £20,000


 50%|█████     | 2/4 [01:45<01:44, 52.39s/it]INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.6 to cache
INFO:policyengine.utils.data.caching_google_storage_client:Cache exists and crc is unchanged for policyengine-uk-data-private, enhanced_frs_2022_23.h5.
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5


{'gov.dwp.universal_credit.elements.child.limit.child_count': 90, 'gov.hmrc.income_tax.allowances.personal_allowance.amount': 20000, 'gov.hmrc.income_tax.rates.uk[1].threshold': 50000} Set the higher rate threshold to £70,000


 75%|███████▌  | 3/4 [02:36<00:51, 51.83s/it]INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.6 to cache
INFO:policyengine.utils.data.caching_google_storage_client:Cache exists and crc is unchanged for policyengine-uk-data-private, enhanced_frs_2022_23.h5.
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5


{'gov.dwp.universal_credit.elements.child.limit.child_count': 90, 'gov.hmrc.income_tax.allowances.personal_allowance.amount': 20000, 'gov.hmrc.income_tax.rates.uk[1].threshold': 50000, 'gov.hmrc.income_tax.allowances.marriage_allowance.max': 0.25} Raise the Marriage Allowance rate to 25%


100%|██████████| 4/4 [03:26<00:00, 51.74s/it]


In [65]:
import pandas as pd

df = pd.DataFrame({
    "Year": years,
    "Debt impact (£ billions)": debt_impacts,
    "Policy": policies,
})

df = df.pivot(columns="Year", index="Policy", values="Debt impact (£ billions)")
df = df.iloc[[2, 1, 3, 0]]

In [67]:
df.values[1:] = df.values[1:] - df.values[:-1]

In [68]:
df["2025-29"] = df.sum(axis=1)

In [70]:
# Add 'total' row
df.loc["Total"] = df.sum(numeric_only=True)

In [71]:
print(df.round(1).to_markdown(index=True))

| Policy                                   |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-29 |
|:-----------------------------------------|-------:|-------:|-------:|-------:|-------:|----------:|
| Repeal the two-child limit               |   -1.7 |   -1.8 |   -1.9 |   -2.2 |   -2.3 |      -9.9 |
| Raise the Personal Allowance to £20,000  |  -50.7 |  -51.5 |  -52.4 |  -49.1 |  -45.5 |    -249.3 |
| Set the higher rate threshold to £70,000 |  -22.2 |  -23.8 |  -25.5 |  -27.2 |  -29.3 |    -128   |
| Raise the Marriage Allowance rate to 25% |   -0.7 |   -0.7 |   -0.8 |   -0.8 |   -0.9 |      -3.9 |
| Total                                    |  -75.3 |  -77.9 |  -80.5 |  -79.3 |  -78.1 |    -391.1 |
